In [9]:
import random
class Compte:
    def __init__(self, numero: int, nom: str):
        self.numero = numero
        self.nom = nom
        self.solde = 0.0
        self.historique = []
        self.transferts = {} 
        self.pret_en_cours = 0.0
        self.pret_initial = 0.0
        self.compte_preferentiel = None

    def ajouter(self, montant: float):
        self.solde += montant
        self.historique.append(f"Ajout de {montant:.2f} €")
        if self.pret_en_cours > 0:
            remboursement = min(self.solde, self.pret_en_cours * 0.3 * 1.1)
            self.solde -= remboursement
            self.pret_en_cours -= remboursement / 1.1
            self.historique.append(f"Remboursement auto de {remboursement:.2f} €")
            if self.pret_en_cours <= 0:
                self.historique.append("Prêt remboursé.")
                self.pret_en_cours = 0.0
                self.pret_initial = 0.0

    def retirer(self, montant: float) -> bool:
        if self.solde >= montant:
            self.solde -= montant
            self.historique.append(f"Retrait de {montant:.2f} €")
            return True
        else:
            self.historique.append("Retrait refusé : solde insuffisant")
            return False

    def transfert(self, montant: float, autre_compte: 'Compte'):
        if self.pret_en_cours > 0:
            self.historique.append("Transfert refusé : prêt en cours")
            return

        if self.retirer(montant):
            autre_compte.ajouter(montant)
            self.transferts[autre_compte.nom] = self.transferts.get(autre_compte.nom, 0) + montant
            self.historique.append(f"Transfert de {montant:.2f} € vers {autre_compte.nom}")

    def demander_pret(self, montant: float):
        if self.pret_en_cours == 0:
            self.pret_en_cours = montant
            self.pret_initial = montant
            self.solde += montant
            self.historique.append(f"Prêt reçu de {montant:.2f} €")

    def afficher(self):
        print(f"[COMPTE] {self.nom} | Solde: {self.solde:.2f} € | Prêt restant: {self.pret_en_cours:.2f} €")

    def afficher_historique(self):
        print("[HISTORIQUE]")
        for h in self.historique:
            print(" -", h)


class AgentIntelligent:
    def __init__(self, compte: Compte):
        self.compte = compte
        self.actions = 0
        self.memoire_transfert = {}  

    def definir_destinataire_principal(self, compte: Compte):
        self.compte.compte_preferentiel = compte

    def percevoir_et_agir(self):
        solde = self.compte.solde
        pret = self.compte.pret_en_cours
        self.actions += 1

        print("[AGENT] Évaluation du contexte bancaire...")

        if solde < 50 and pret == 0:
            self.compte.demander_pret(200)

        elif self.actions % 3 == 0 and pret == 0 and self.compte.compte_preferentiel:
            nom_dest = self.compte.compte_preferentiel.nom
            freq = self.memoire_transfert.get(nom_dest, 0)
            if freq >= 2 and solde > 100:
                montant = min(50, solde - 50)
                self.compte.transfert(montant, self.compte.compte_preferentiel)
                self.memoire_transfert[nom_dest] += 1

        self._apprentissage_transfert()
        self.compte.afficher()
        self.compte.afficher_historique()
        print("=" * 40)

    def _apprentissage_transfert(self):
        for nom, montant in self.compte.transferts.items():
            if nom in self.memoire_transfert:
                self.memoire_transfert[nom] += 1
            else:
                self.memoire_transfert[nom] = 1

principal = Compte(1001, "Client Principal")
loyer = Compte(2002, "Compte Loyer")

agent = AgentIntelligent(principal)
agent.definir_destinataire_principal(loyer)
principal.ajouter(30)
agent.percevoir_et_agir()
principal.ajouter(100)
agent.percevoir_et_agir()

principal.transfert(20, loyer)
principal.transfert(30, loyer)

principal.ajouter(150)
agent.percevoir_et_agir()

principal.ajouter(200)
agent.percevoir_et_agir()  

loyer.afficher()
loyer.afficher_historique()


[AGENT] Évaluation du contexte bancaire...
[COMPTE] Client Principal | Solde: 230.00 € | Prêt restant: 200.00 €
[HISTORIQUE]
 - Ajout de 30.00 €
 - Prêt reçu de 200.00 €
[AGENT] Évaluation du contexte bancaire...
[COMPTE] Client Principal | Solde: 264.00 € | Prêt restant: 140.00 €
[HISTORIQUE]
 - Ajout de 30.00 €
 - Prêt reçu de 200.00 €
 - Ajout de 100.00 €
 - Remboursement auto de 66.00 €
[AGENT] Évaluation du contexte bancaire...
[COMPTE] Client Principal | Solde: 367.80 € | Prêt restant: 98.00 €
[HISTORIQUE]
 - Ajout de 30.00 €
 - Prêt reçu de 200.00 €
 - Ajout de 100.00 €
 - Remboursement auto de 66.00 €
 - Transfert refusé : prêt en cours
 - Transfert refusé : prêt en cours
 - Ajout de 150.00 €
 - Remboursement auto de 46.20 €
[AGENT] Évaluation du contexte bancaire...
[COMPTE] Client Principal | Solde: 535.46 € | Prêt restant: 68.60 €
[HISTORIQUE]
 - Ajout de 30.00 €
 - Prêt reçu de 200.00 €
 - Ajout de 100.00 €
 - Remboursement auto de 66.00 €
 - Transfert refusé : prêt en cour